# Exercise 1

In [ ]:
import pandas as pd
df = pd.read_csv("2023_01.csv.zip")
df

,coordsx,coordsy,created,observed,parameterId,stationId,value
0,9.7875,56.1496,2023-07-07T21:57:06.803045Z,2023-01-17T23:59:00Z,precip_past1min,5185,0.0
1,8.1465,56.5032,2023-07-07T21:57:44.125851Z,2023-01-17T23:59:00Z,precip_past1min,5296,0.0
2,9.2877,57.0143,2023-07-07T21:56:34.840129Z,2023-01-17T23:59:00Z,precip_past1min,5085,0.0
3,8.6652,54.9872,2023-07-07T21:58:08.833412Z,2023-01-17T23:59:00Z,precip_past1min,5355,0.0
4,12.0981,55.2076,2023-07-07T21:59:11.626531Z,2023-01-17T23:59:00Z,precip_past1min,5889,0.0
...,...,...,...,...,...,...,...
8142490,14.7494,55.0677,2023-07-08T07:41:47.665513Z,2023-01-01T00:00:00Z,wind_dir_past1h,6190,209.0
8142491,-51.7308,64.1833,2023-07-08T07:41:47.637314Z,2023-01-01T00:00:00Z,temp_min_past1h,4250,-9.2
8142492,10.1272,56.3027,2023-07-07T17:20:39.101115Z,2023-01-01T00:00:00Z,precip_past10min,6072,0.5
8142493,9.4041,56.8790,2023-07-07T20:15:09.826122Z,2023-01-01T00:00:00Z,precip_dur_past10min,5081,10.0


# Exercise 2 saving time in datatime64 and changing to category for repeated items save more space 

In [ ]:

def reduce_dmi_df(df):
    df['created'] = pd.to_datetime(df['created'], format="ISO8601", utc=True)
    df['observed'] = pd.to_datetime(df['observed'], format="ISO8601", utc=True)
    df['parameterId'] = df['parameterId'].astype('category')
    df['stationId'] = df['stationId'].astype('int32')
    return df

reduce_dmi_df(df)

,coordsx,coordsy,created,observed,parameterId,stationId,value
0,9.7875,56.1496,2023-07-07 21:57:06.803045+00:00,2023-01-17 23:59:00+00:00,precip_past1min,5185,0.0
1,8.1465,56.5032,2023-07-07 21:57:44.125851+00:00,2023-01-17 23:59:00+00:00,precip_past1min,5296,0.0
2,9.2877,57.0143,2023-07-07 21:56:34.840129+00:00,2023-01-17 23:59:00+00:00,precip_past1min,5085,0.0
3,8.6652,54.9872,2023-07-07 21:58:08.833412+00:00,2023-01-17 23:59:00+00:00,precip_past1min,5355,0.0
4,12.0981,55.2076,2023-07-07 21:59:11.626531+00:00,2023-01-17 23:59:00+00:00,precip_past1min,5889,0.0
...,...,...,...,...,...,...,...
8142490,14.7494,55.0677,2023-07-08 07:41:47.665513+00:00,2023-01-01 00:00:00+00:00,wind_dir_past1h,6190,209.0
8142491,-51.7308,64.1833,2023-07-08 07:41:47.637314+00:00,2023-01-01 00:00:00+00:00,temp_min_past1h,4250,-9.2
8142492,10.1272,56.3027,2023-07-07 17:20:39.101115+00:00,2023-01-01 00:00:00+00:00,precip_past10min,6072,0.5
8142493,9.4041,56.8790,2023-07-07 20:15:09.826122+00:00,2023-01-01 00:00:00+00:00,precip_dur_past10min,5081,10.0


In [26]:
import numpy as np
print(np.abs(df['stationId'] -df['stationId'].astype('int64')).sum())
print(np.abs(df['stationId'] -df['stationId'].astype('int64')).max())

0
0


In [ ]:
def summarize_columns(df):
    print(pd.DataFrame([
        (
            c,
            df[c].dtype,
            len(df[c].unique()),
            df[c].memory_usage(deep=True) // (1024**2)
        ) for c in df.columns
    ], columns=['name', 'dtype', 'unique', 'size (MB)']))
    print('Total size:', df.memory_usage(deep=True).sum() / 1024**2, 'MB')
    


          name                dtype   unique  size (MB)
0      coordsx              float64      224         62
1      coordsy              float64      219         62
2      created  datetime64[ns, UTC]  8142495         62
3     observed  datetime64[ns, UTC]    44640         62
4  parameterId             category       47          7
5    stationId                int64      247         62
6        value              float64    11532         62
Total size: 380.503137588501 MB


# 3&4 pyarrow load quicker

In [36]:
from pyarrow import csv
def pyarrow_load(fname):
    table = csv.read_csv(fname)
    df = table.to_pandas()
    return df
from time import perf_counter
Start = perf_counter()
df = pyarrow_load("2023_01.csv")
summarize_columns(df)
print(perf_counter()-Start )

Start = perf_counter()
df = pd.read_csv("2023_01.csv")
summarize_columns(df)
print( perf_counter() - Start)

          name                dtype   unique  size (MB)
0      coordsx              float64      224         62
1      coordsy              float64      219         62
2      created  datetime64[ns, UTC]  8142495         62
3     observed   datetime64[s, UTC]    44640         62
4  parameterId               object       47        484
5    stationId                int64      247         62
6        value              float64    11532         62
Total size: 857.0067491531372 MB
5.333899582969025
          name    dtype   unique  size (MB)
0      coordsx  float64      224         62
1      coordsy  float64      219         62
2      created   object  8142495        590
3     observed   object    44640        535
4  parameterId   object       47        484
5    stationId    int64      247         62
6        value  float64    11532         62
Total size: 1858.7288799285889 MB
23.513865750050172


# 5. save to parquet file save space